dataset_chap123.ipynb

    각 데이터를 다루는 클래스들을 살펴보며, 정의된 클래스들 담아주도록 한다.

In [1]:
import os

os.chdir('C:\\Users\\gowra\\1_멋쟁이 사자처럼\\2. 신경망 기법\\신경망(클래스_프로그램_제작_프로젝트)/code/')

In [2]:
%run dataset.ipynb

#### 전복의 고리 수를 추정하는 abalone 데이터셋을 다루는 AbaloneDataset() 클래스

In [ ]:
class AbaloneDataset(Dataset):
    def __init__(self):
        super(AbaloneDataset, self).__init__('abalone', 'regression')
    
        rows, _ = load_csv('C:/Users/gowra/1_멋쟁이 사자처럼/2. 신경망 기법/data/abalone.csv')

        xs = np.zeros([len(rows), 10])  # 행, 열(독립변수)
        ys = np.zeros([len(rows), 1])  # 행, 열(종속변수)

        for n, row in enumerate(rows):
            if row[0] == 'I': xs[n, 0] = 1
            if row[0] == 'M': xs[n, 1] = 1
            if row[0] == 'F': xs[n, 2] = 1
            xs[n, 3:] = row[1:-1]  # 종속변수를 제외한 변수들을 불러온다.
            ys[n, :] = row[-1:]  # 종속변수만 불러온다.
        
        self.shuffle_data(xs, ys, 0.8)

    def visualize(self, xs, estimates, answers):  # 실제값, 추정값, 정답
        for n in range(len(xs)):
            x, est, ans = xs[n], estimates[n], answers[n]
            xstr = vector_to_str(x, '%4.2f')
            print('{} => 추정 {:4.1f} : 정답 {:4.1f}'.
                  format(xstr, est[0], ans[0]))

    Dataset()을 상속받는 과정에서 데이터의 이름인 name과 어떠한 모드인지를 알려주는 mode 변수를 각각 'abalone', 'regression' 로 설정
    Dataset() 클래스에서 mode를 전달받는 메서드들은 자식 클래스에서 전달되어진 name과 mode에 맞춰 기능을 수행

    파일을 불러오는 과정인 load_csv()를 통해 csv 데이터셋 파일을 불러옵니다.
    load_csv() 메서드는 이후 살펴볼 mathutil.ipynb 에서 정의되는데, 데이터셋의 rows와 header를 반환
    만약 헤더정보가 없더라도 데이터셋을 활용할 수 있도록 하였으며, 이후 다중 선택 분류 과정에서 다수의 헤더정보를 정답 라벨로 간편하게 설정할 수 있게 설계

    데이터를 읽어들였으니 np.zeros() 를 통해 독립 변수와 종속 변수의 버퍼를 생성
    버퍼에 읽어들인 데이터를 복사하는 과정인데, abalone 데이터의 경우 문자열이 독립 변수에 포함되어 있습니다. 
    그래서 이를 표현하기 위해 원-핫 벡터 표현으로 전환하며, 이후의 변수들 또한 모두 복사하여 xs, ys에 저장

    생성된 xs, ys는 Dataset() 클래스에서 정의한 shuffle_data() 메서드를 호출해 학습 / 검증 / 테스트 데이터셋으로 분할하고
    데이터에 대한 입출력 벡터를 챙겨준다.

```python
# mlp_model.ipynb
class Model(object):
    def exec_all(self,
                 epoch_count=10,
                 batch_size=10,
                 learning_rate=0.001,
                 report=0,
                 show_cnt=3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0:
            self.visualize(show_cnt)


# mlp_model.ipynb
def mlp_model_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    self.dataset.visualize(deX, est, deY)  # AbaloneDataset(Dataset): 의 visualize를 수행


MlpModel.visualize = mlp_model_visualize


# dataset_chap123.ipynb
class AbaloneDataset(Dataset):
    def init(self):
         ...

    def visualize(self, xs, estimates, answers):
        for n in range(len(xs)):
            x, est, ans = xs[n], estimates[n], answers[n]
            xstr = vector_to_str(x, '%4.2f')
            print('{} => 추정 {:4.1f} : 정답 {:4.1f}'.format(xstr, est[0], ans[0]))
            
```

    다음 메서드의 경우 검증 데이터를 통해 예측을 수행하고 실제 결과값을 비교하여 출력하는 기능을 수행합니다. 
    visualize() 메서드의 경우 실행 순서가 약간 복잡하지만 천천히 살펴  보겠습니다.

    우선 Model() 클래스의 exec_all() 메서드가 실행될 때 가장 마지막 메서드인 visualize() 메서드의 매개 변수인 show_cnt가 1 이상인 경우와 그 값 만큼 결과를 출력하게 됩니다. 
    이러한 visualize() 메서드는 Model() 클래스를 상속받은 MlpModel() 클래스에서 정의되었는데, 
    mlp_model_visualize() 메서드에는 get_visualize_data(num) 메서드가 동작하며 num으로 전달된 값 만큼 deX, deY를 출력합니다.
    이러한 상황에서 self.dataset.visualize(deX, est, deY) 메서드에 앞서 구한 매개 변수들이 전달되어 집니다. 

    그리고 Dataset() 클래스를 상속받은  AbaloneDataset() 클래스의 visualize() 메서드는 전달 받은 변수들을 통해 show_cnt 값 만큼의 데이터가 순차적으로 반복문을 타며 x, est, ans로 저장됩니다.
    그리고 vector_to_str()메서드에 x와 출력 포멧팅 설정값을 넣어주면 입력 데이터 x가  문자열로 하여 저장되어 집니다. 
    vector_to_str()는 이후 살펴보도록 하겠습니다.

    마지막으로 print()를 활용하여 입력 데이터 문자열, 신경망 추측, 정답을 차례로 넣어 show_cnt 만큼 출력을 수행하도록 합니다. 
    그럼 이후에 MlpModel() 클래스를 객체화 시킨 후, 그 객체의 exec_all() 혹은 visualize() 를 통해 결과를 아래와 같이 확인할 수 있게 됩니다.


#### PulsarDataset() 클래스

In [ ]:
class PulsarDataset(Dataset):
    def __init__(self):
        super(PulsarDataset, self).__init__('pulsar', 'binary')
    
        rows, _ = load_csv('C:/Users/gowra/1_멋쟁이 사자처럼/2. 신경망 기법/data/pulsar_stars.csv')

        data = np.asarray(rows, dtype='float32')
        self.shuffle_data(data[:,:-1], data[:,-1:], 0.8) # 학습 0.8 / 테스트 
        self.target_names = ['별', '펄서']  # target을 설정
        
    def visualize(self, xs, estimates, answers):
        for n in range(len(xs)):
            x, est, ans = xs[n], estimates[n], answers[n]
            xstr = vector_to_str(x, '%5.1f', 3)
            estr = self.target_names[int(round(est[0]))]
            astr = self.target_names[int(round(ans[0]))]
            rstr = 'O'
            if estr != astr: rstr = 'X'
            print('{} => 추정 {}(확률 {:4.2f}) : 정답 {} => {}'. \
                  format(xstr, estr, est[0], astr, rstr))

    Dataset() 을 상속받는 과정에서 name 과 mode에 대하여 각 'pulsar', 'binary' 값으로 설정하며, 
    load_csv() 메서드를 통해 데이터를 읽어오고 shuffle_data() 메서드로 데이터를 섞어주며 각각 학습/검증/  평가를 위해 나눠 줍니다.

    다음으로 이 문제의 경우 데이터의 정답이 'stars' 와 'pulsar'로 나뉜 이진 분류입니다. 
    그래서 결과 출력을 위해 target_names 에 문자열 '별'과 '펄서'를 리스트 형식으로 넣어두게 되면 '별'은 0, '펄서'는 1의 값을 가지게 되는데, 
    이는 데이터셋의 실제 종속 변수와 동일한 값을 갖게되는 것이며, 
    이를 통해  하단의 visualize() 메서드에서 조금 더 직관적인 결과를 출력할 수 있습니다.


```python
# mlp_model.ipynb
class Model(object):
    def exec_all(self,
                 epoch_count=10,
                 batch_size=10,
                 learning_rate=0.001,
                 report=0,
                 show_cnt=3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0:
            self.visualize(show_cnt)


# mlp_model.ipynb
def mlp_model_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    self.dataset.visualize(deX, est, deY)


MlpModel.visualize = mlp_model_visualize


# dataset_chap123.ipynb
class PulsarDataset(Dataset):
    def init__(self):
        …

        
    # Model(object): exec_all()의 show_cnt 만큼 가져온다
    def visualize(self, xs, estimates, answers): 
        for n in range(len(xs)):
            x, est, ans = xs[n], estimates[n], answers[n]
            xstr = vector_to_str(x, '%5.1f', 3)
            estr = self.target_names[int(round(est[0]))]
            astr = self.target_names[int(round(ans[0]))]
            rstr = 'O'
            if estr != astr: rstr = 'X'   # 추정값과 정답값이 같지 않다면
            print('{} => 추정 {}(확률 {:4.2f}) : 정답 {} => {}'.format(
                xstr, estr, est[0], astr, rstr))
            
```           

    visualize() 메서드의 경우도 위와 마찬가지로 Model() 클래스의  exec_all() 메서드에서 visualize() 메서드가 실행되면서 
    이 메서드 내부의  self.dataset.visualize()가 수행되는 구조입니다. 

    여기서 Dataset() 클래  스에서 바로 동작하는 것이 아닌, Dataset() 클래스를 상속받는 클래스에서 정의되는 이유는 문제의 성격에 따라 다루는 데이터가 달라지기 때문입니다.  
    이 메서드의 기능은 역시 시각화를 수행하게 되는데, 앞서 받아온 입력 값, 신경망 예측 값, 실제 정답을 반복문 for 를 통해 차례로 입력받아줍니다.

    그리고 출력을 위한 xstr, estr, astr 세 가지를 정의할 것인데, 우선 xstr은  vector_to_str() 메서드를 통해 얻어 줄 것 입니다.
    이 메서드는 출력할 입력 벡터 x, 출력 포멧팅 설정 값, 입력 벡터의 최대 출력수 이렇게 세 가지를 전달받으며, 이에 맞춰 반환값을 xstr에 저장합니다.
    만약 입력 벡터의 수가 최대 출력 수를 넘어가게 되면 최대 출력 수 이하의 값  은 '...' 처리되어 출력되게끔 설계됩니다.

    이후 살펴보겠지만 최대 출력 수는 설계자의 선택에 따라 조절할 수 있습니다.  
    다음으로 estr, astr의 경우 활성화 함수 시그모이드를 거친 추측과 실제 정답의 결과를 원활하게 비교하기 위하여 각각 반올림 그리고 정수화를 진행 후 결과값을 저장합니다.

    그리고 이 두개의 값을 비교한 결과를 담을 수 있는 rstr을 생성하여 estr과 astr이 같이 않다면 rstr을 'X'로 저장합니다.

    최종 출력을 위한 print()는 이러한 결과를 담을 수 있도록 작성하며 메서드 정의를 마칩니다.


#### 다중 선택 분류 데이터를 다루는 SteelDataset() 클래스

In [ ]:
class SteelDataset(Dataset):
    def __init__(self):
        super(SteelDataset, self).__init__('steel', 'select')
    
        rows, headers = load_csv('C:/Users/gowra/1_멋쟁이 사자처럼/2. 신경망 기법/data/faults.csv')

        data = np.asarray(rows, dtype='float32')
        self.shuffle_data(data[:,:-7], data[:,-7:], 0.8)
        
        self.target_names = headers[-7:]
        
    def visualize(self, xs, estimates, answers):
        show_select_results(estimates, answers, self.target_names) # mathutil에 정의

    이 데이터셋의 경우 정답 라벨이 다소 많은 축에 속합니다. 
    그래서 하나씩 값을 넣어주기 보다는 데이터셋의 종속변수에 해당하는 헤더정보를 self.target_naems 에 할당하게 되면 번거로움 없이 코드를 작성할 수 있다.

    SteelDataset() 클래스도 마찬가지로 문제의 성격에 맞는 visualize() 메서드가 정의됩니다.
    여기서는 show_select_results() 메서드가 실행되어 지는데, 
    이 메서드의 실행 결과로는 데이터 처리 결과에 따른 추정확률 분포, 
    추정확률 분포에 따른선택결과, 실제 정답, 일치 여부 이렇게 네 가지의 결과가 출력되게 설계되어 있습니다.

    이 메서드는 이후 mathutil.ipynb 파일에서 천천히 살펴보겠습니다.


#### 이진 분류 문제를 다중 선택 분류 문제로 

In [ ]:
class PulsarSelectDataset(Dataset):
    def __init__(self):
        super(PulsarSelectDataset, self).__init__('pulsarselect', 'select')
    
        rows, _ = load_csv('C:/Users/gowra/1_멋쟁이 사자처럼/2. 신경망 기법/data/pulsar_stars.csv')

        data = np.asarray(rows, dtype='float32')
        self.shuffle_data(data[:,:-1], onehot(data[:,-1], 2), 0.8)
        self.target_names = ['별', '펄서']
        
    def visualize(self, xs, estimates, answers):
        show_select_results(estimates, answers, self.target_names)

    단, shuffle_data() 메서드에서 종속 변수 부분을 하나의 값을 갖는 이진 분류 타입으로 넘겨주지 않고, 
    이 부분을 onehot() 메서드를 통해 다중 선택 분류 처럼 값을 넘겨주면 됩니다. 
    onehot(data[:,-1], 2)  >  종속변수의 종류 = 2개

    그리고 시각화를 위한 visualize()메서드 또한 수정없이 그대로 사용하여 시각화를 수행할 수 있습니다.
    마찬가지로 이 onehot() 또한 이후 mathutil.ipynb에서 정의하도록 하겠습니다.
